In [2]:
users = UserDB.query.all()

print(len(users))
for user in users:
    print(user)

0


In [3]:
db.create_all()

In [2]:
def df2studyeventsdb(df):
    study_events = []
    for i, row in df.iterrows():
        study_events.append(StudyEventDB(username=row[NAME],
                                         date=date2datetime(row[DATE]),
                                         start_time=row[START_TIME],
                                         end_time=row[END_TIME]))
    return study_events


def studyeventsdb2df(db):
    study_events = StudyEventDB.query.all()

    df_dict = {}
    df_dict[NAME] = [se.username for se in study_events]
    df_dict[DATE] = [datetime2date(se.date) for se in study_events]
    df_dict[START_TIME] = [se.start_time for se in study_events]
    df_dict[END_TIME] = [se.end_time for se in study_events]

    return pd.DataFrame(df_dict)

In [2]:
gs = GoogleSheet.read_from(STUDY_TIME_TABLE_NAME)

In [3]:
df_dur = gs.sheet(sheet_name=SHEET1, least_col_name=START_TIME)
df_eve = gs.sheet(sheet_name=SHEET2, least_col_name=NAME)

df = merge_dur_eve(df_dur, df_eve)

In [4]:
df.head()

,name,date,start_time,end_time
0,Alpha,2020.11.09,11:00,12:00
1,Delta,2020.11.09,11:00,12:00
2,Delta,2020.11.09,08:00,15:00
3,Beta,2020.11.09,08:00,15:00
4,Diqing,2020.11.09,19:00,23:00


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 338 entries, 0 to 186
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        338 non-null    object
 1   date        338 non-null    object
 2   start_time  338 non-null    object
 3   end_time    338 non-null    object
dtypes: object(4)
memory usage: 13.2+ KB


In [2]:
 # delete study events
StudyEventDB.query.delete()
db.session.commit()

# load df to db
study_events = df2studyeventsdb(df)
db.session.add_all(study_events)
db.session.commit()

NameError: name 'df2studyeventsdb' is not defined

In [12]:
study_events = StudyEventDB.query.all()

df_dict = {}
df_dict[NAME] = [se.username for se in study_events]
df_dict[DATE] = [datetime2date(se.date) for se in study_events]
df_dict[START_TIME] = [datetime2time(se.start_time) for se in study_events]
df_dict[END_TIME] = [datetime2time(se.end_time) for se in study_events]

df = pd.DataFrame(df_dict)
df.head()

,name,date,start_time,end_time
0,Alpha,2020.11.09,11:00,12:00
1,Delta,2020.11.09,11:00,12:00
2,Delta,2020.11.09,08:00,15:00
3,Beta,2020.11.09,08:00,15:00
4,Diqing,2020.11.09,19:00,23:00


In [13]:
df=df.append({'name': "Congyu", "date": "2021.01.17", "start_time": "11:18"}, ignore_index=True)
df.tail()

,name,date,start_time,end_time
334,LiuYang,2020.12.28,09:17,09:47
335,vivi,2021.01.05,13:23,13:53
336,Ye Liu,2021.01.15,20:23,20:53
337,Congyu,2021.01.17,10:50,11:20
338,Congyu,2021.01.17,11:18,NaN


In [24]:
df.tail()

,name,date,start_time,end_time
334,LiuYang,2020.12.28,09:17,09:47
335,vivi,2021.01.05,13:23,13:53
336,Ye Liu,2021.01.15,20:23,20:53
337,Congyu,2021.01.17,10:50,11:20
338,Congyu,2021.01.17,11:18,NaN


In [6]:
gs = GoogleSheet.read_from(STUDY_TIME_TABLE_NAME)
df = gs.sheet(sheet_name=SHEET3)
study_events = df2studyeventsdb(df)

# delete study events
StudyEventDB.query.delete()
db.session.commit()

# load df to db
db.session.add_all(study_events)
db.session.commit()

In [7]:
df = studyeventsdb2df(db)
df.head(10)

,name,date,start_time,end_time
0,Congyu,2021.01.17,11:38,11:44
1,Shangsu,2021.01.17,11:38,11:44
2,Congyu,2021.01.17,11:38,11:45
3,Shangsu,2021.01.17,11:48,11:49
4,Congyu,2021.01.17,11:49,11:49
5,Congyu,2021.01.17,12:19,12:19
6,Congyu,2021.01.17,12:19,unknown


In [10]:
for i, row in df.iterrows():
    print(row.tolist())

['Congyu', '2021.01.17', '11:38', '11:44']
['Shangsu', '2021.01.17', '11:38', '11:44']
['Congyu', '2021.01.17', '11:38', '11:45']
['Shangsu', '2021.01.17', '11:48', '11:49']
['Congyu', '2021.01.17', '11:49', '11:49']
['Congyu', '2021.01.17', '12:19', '12:19']
['Congyu', '2021.01.17', '12:19', 'unknown']


In [3]:
import time
from tqdm import tqdm


In [4]:

gs = GoogleSheet.read_from(STUDY_TIME_TABLE_NAME)
df_dur = gs.sheet(sheet_name=SHEET1, least_col_name=START_TIME)
df_eve = gs.sheet(sheet_name=SHEET2, least_col_name=NAME)

df = merge_dur_eve(df_dur, df_eve)

In [8]:


gs = GoogleSheet.read_from(STUDY_TIME_TABLE_NAME)
for i, row in tqdm(df.iterrows(), total=len(df)):
    if i > 81-2:
        gs.append_row(SHEET3, row.tolist())
        time.sleep(1.001)

100%|██████████| 338/338 [05:00<00:00,  1.13it/s]


In [35]:
df=df.append({'name': "Congyu", "date": "2021.01.17", "start_time": "11:18", "end_time": UNKNOWN}, ignore_index=True)
df.tail()

,name,date,start_time,end_time,start_time_datetime,end_time_datetime,minutes,date_datetime,weekday
334,LiuYang,2020.12.28,09:17,09:47,1900-01-01 09:17:00,1900-01-01 09:47:00,30.0,2020-12-28,Monday
335,vivi,2021.01.05,13:23,13:53,1900-01-01 13:23:00,1900-01-01 13:53:00,30.0,2021-01-05,Tuesday
336,Ye Liu,2021.01.15,20:23,20:53,1900-01-01 20:23:00,1900-01-01 20:53:00,30.0,2021-01-15,Friday
337,Congyu,2021.01.17,10:50,11:20,1900-01-01 10:50:00,1900-01-01 11:20:00,30.0,2021-01-17,Sunday
338,Congyu,2021.01.17,11:18,unknown,NaT,NaT,NaN,NaT,NaN


In [19]:
data = df
data.tail()

,name,date,start_time,end_time
336,Ye Liu,2021.01.15,20:23,20:53
337,Congyu,2021.01.17,10:50,11:20
338,Congyu,2021.01.17,11:18,NaN
339,Congyu,2021.01.17,11:18,unkonwn
340,Congyu,2021.01.17,11:18,unknown


In [22]:
df_user = data.loc[data[NAME]=="Congyu", :]
df_user.loc[df_user[END_TIME] == UNKNOWN, START_TIME].tolist()[-1]

'11:18'

In [25]:
df = add_analysis_columns(df)
df

,name,date,start_time,end_time,start_time_datetime,end_time_datetime,minutes,date_datetime,weekday
0,Alpha,2020.11.09,11:00,12:00,1900-01-01 11:00:00,1900-01-01 12:00:00,60.0,2020-11-09,Monday
1,Delta,2020.11.09,11:00,12:00,1900-01-01 11:00:00,1900-01-01 12:00:00,60.0,2020-11-09,Monday
2,Delta,2020.11.09,08:00,15:00,1900-01-01 08:00:00,1900-01-01 15:00:00,420.0,2020-11-09,Monday
3,Beta,2020.11.09,08:00,15:00,1900-01-01 08:00:00,1900-01-01 15:00:00,420.0,2020-11-09,Monday
4,Diqing,2020.11.09,19:00,23:00,1900-01-01 19:00:00,1900-01-01 23:00:00,240.0,2020-11-09,Monday
...,...,...,...,...,...,...,...,...,...
182,Hanyu,2020.12.01,11:25,11:55,1900-01-01 11:25:00,1900-01-01 11:55:00,30.0,2020-12-01,Tuesday
183,LiuYang,2020.12.28,09:17,09:47,1900-01-01 09:17:00,1900-01-01 09:47:00,30.0,2020-12-28,Monday
184,vivi,2021.01.05,13:23,13:53,1900-01-01 13:23:00,1900-01-01 13:53:00,30.0,2021-01-05,Tuesday
185,Ye Liu,2021.01.15,20:23,20:53,1900-01-01 20:23:00,1900-01-01 20:53:00,30.0,2021-01-15,Friday


In [3]:
df = studyeventsdb2df(db)
df

,name,date,start_time,end_time
0,Alpha,2020.11.09,11:00,12:00
1,Delta,2020.11.09,11:00,12:00
2,Delta,2020.11.09,08:00,15:00
3,Beta,2020.11.09,08:00,15:00
4,Diqing,2020.11.09,19:00,23:00
...,...,...,...,...
265,Amber,2021.01.18,23:31,23:59
266,L.path,2021.01.18,09:37,unknown
267,Amber,2021.01.18,09:41,unknown
268,Congyu,2021.01.18,09:41,unknown


In [21]:

gs = GoogleSheet.read_from(STUDY_TIME_TABLE_NAME)
df_dur = gs.sheet(sheet_name=SHEET1, least_col_name=START_TIME)
df_eve = gs.sheet(sheet_name=SHEET2, least_col_name=NAME)

df = merge_dur_eve(df_dur, df_eve)


StudyEventDB.query.delete()
db.session.commit()

# load df to db
study_events = df2studyeventsdb(df)
db.session.add_all(study_events)
db.session.commit()
# for study_event in study_events:
#    db.session.add(study_event)
#    db.session.commit()

df2 = studyeventsdb2df(db)


KeyboardInterrupt: 

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 142
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        284 non-null    object
 1   date        284 non-null    object
 2   start_time  284 non-null    object
 3   end_time    284 non-null    object
dtypes: object(4)
memory usage: 11.1+ KB


In [21]:
df = df.reset_index(drop=True)

In [22]:
df2 = df2.reset_index(drop=True)

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        284 non-null    object
 1   date        284 non-null    object
 2   start_time  284 non-null    object
 3   end_time    284 non-null    object
dtypes: object(4)
memory usage: 9.0+ KB


In [39]:
import numpy as np
np.array_equal(df.values, df2.values)

print(df.values.shape)
print(df2.values.shape)

print(df.values)
print()
print("-"*50)
print()
print(df2.values)

(284, 4)
(284, 4)
[['Alpha' '2020.11.09' '11:00' '12:00']
 ['Delta' '2020.11.09' '11:00' '12:00']
 ['Delta' '2020.11.09' '08:00' '15:00']
 ...
 ['vivi' '2021.01.05' '13:23' '13:53']
 ['Shangsu' '2021.01.11' '10:07' '10:37']
 ['Congyu' '2021.01.11' '09:15' '09:45']]

--------------------------------------------------

[['Alpha' '2020.11.09' '11:00' '12:00']
 ['Delta' '2020.11.09' '11:00' '12:00']
 ['Delta' '2020.11.09' '08:00' '15:00']
 ...
 ['vivi' '2021.01.05' '13:23' '13:53']
 ['Shangsu' '2021.01.11' '10:07' '10:37']
 ['Congyu' '2021.01.11' '09:15' '09:45']]


In [45]:
for i, j in zip(df.values.ravel(), df2.values.ravel()):
    if i!=j:
        print("{}!={}".format(i, j))
        print(type(i))
        print(type(j))
        print(len(i))
        print(len(j))

8:30!=08:30
<class 'str'>
<class 'str'>
4
5
9:30!=09:30
<class 'str'>
<class 'str'>
4
5


In [11]:
for i, row in df.iterrows():
    print(row["date"])
    print(type(row["start_time"]))
    break

2020.11.09
<class 'str'>
